# find the sessions that include the region of interest
Note that the function list any session that includes **any** of the regions. The funciton to list the sessions that include **all** of the regions will be added in the future.

In [1]:
from functions import  get_epoch_StimOn, get_pid_eid_pairs
from functions import monitor_job_status
import submitit
import pandas as pd 
import numpy as np
import json 
from pathlib import Path
import os
import sys
sys.path.append(str(Path(os.getcwd()).resolve().parent)) # add the root of the project to the python path
# Now import the paths from config.py
from config import paths

# create the pid_eid_pairs
pid_eid_pairs = get_pid_eid_pairs(output_file='data/pid_eids_V1.json', only_passive= False, regions= ['VISp'])

# the path to the data (based on config file) and the number of downloaded files
print(f'the path to the data is {paths["LFP"]}')
already_done = [p[0] for p in pid_eid_pairs if p[0] in [f.split('_')[0] for f in os.listdir(paths['LFP'])]]
print(f'already done {len(already_done)}')




Connected to https://openalyx.internationalbrainlab.org as user "intbrainlab"
Total number of possible probe insertions: 73
PID-EID pairs saved to data/pid_eids_V1.json
the path to the data is /workspaces/onedrive1/文档/ibl_data
already done 10


# 1 parallel submition using submitit 

In [ ]:
import submitit
import os
import json

def submit_extract(pid_eid_pair):
    pid, eid = pid_eid_pair
    get_epoch_StimOn(pid, modee = 'download', overwrite= False, window_secs=[-1, 1.5], save = True, ephys_path = paths['LFP'])


already_done_pids = [f.split('_')[0] for f in os.listdir(paths['LFP']) if f.endswith('.npy') ]
pid_eid_pair = [p for p in pid_eid_pairs if p[0] not in already_done_pids]
# Create an executor
executor = submitit.AutoExecutor(folder='logs')
executor.update_parameters(slurm_array_parallelism= 50 , mem_gb=10, timeout_min= 120, slurm_partition= 'CPU',  cpus_per_task= 1)
# Submit all jobs using map_array                                                                                              
all_jobs = executor.map_array(submit_extract, pid_eid_pair)
monitor_job_status(all_jobs)

# 2 single submission

In [ ]:
already_done_pids = [f.split('_')[0] for f in os.listdir(paths['LFP']) if f.endswith('.npy') ]
pid_eid_pair = [p for p in pid_eid_pairs if p[0] not in already_done_pids]
for i, pid in enumerate(pid_eid_pair[0]):
    print(pid)
    print(f'{i}/{len(pid_eid_pair)}')
    get_epoch_StimOn(pid, modee = 'download', overwrite= False, window_secs=[-1, 1.5], save = True, ephys_path = paths['LFP'])
    break
    
    

#  NPY to MNE (optional)
due to RAM limitation on local machine, I saved the data in npy format. you can change to fif in this way :

In [ ]:
from one.api import ONE
import os
from functions import get_behavior, get_channels, numpyToMNE
# One-time setup for ONE with cache mode set to 'remote'
ONE.setup(base_url='https://openalyx.internationalbrainlab.org', cache_dir = None,  silent=True)
one = ONE(password='international')
# load data
pid = 'd1a4e8b1-2b6b-4f0d-8b1d-0b3e6f8e8f1d'
epochs_data_np = get_epoch_StimOn(pid, modee = 'download', overwrite= False, window_secs=[-1, 1.5], save = True, ephys_path = paths['LFP'])

# convert npy to mne

eid, label = one.pid2eid(pid)
sfreq =500
behavior = get_behavior(eid, modee='download')
channels = get_channels(eid,pid, modee = 'download')


acronyms = channels['acronym'].tolist()
ch_names=[f'{ch}_{i}' for i, ch in enumerate(acronyms)]
epochs_mne = numpyToMNE(epochs_data_np, behavior, ch_names, sfreq = sfreq )
del epochs_data_np
ephys_path = paths['LFP']
file_path_epochs = os.path.join(ephys_path, f'{pid}_epoch_stimOn.fif')
epochs_mne.save(file_path_epochs, overwrite=True)

Connected to https://openalyx.internationalbrainlab.org as user "intbrainlab"


/opt/conda/envs/iblenvlocal/lib/python3.10/site-packages/one/util.py:543: ALFWarning: Multiple revisions: "", "2024-05-06"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)


Adding metadata with 15 columns
740 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_1667/996204942.py:19: RuntimeWarning: This filename (/workspaces/onedrive1/文档/ibl_data/a8a59fc3-a658-4db4-b5e8-09f1e4df03fd_epoch_stimOn.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_mne.save(file_path_epochs, overwrite=True)


    Writing channel names to FIF truncated to 15 characters with remapping


[PosixPath('/workspaces/onedrive1/文档/ibl_data/a8a59fc3-a658-4db4-b5e8-09f1e4df03fd_epoch_stimOn.fif')]